### Scattering in 1D plane-parallel atmosphere

In [1]:
import numpy 
from scipy.special import roots_legendre
from scipy.sparse import diags

import bqplot.pyplot as plt
from bqplot import LinearScale, LogScale
from ipywidgets import interactive, Layout, HBox, VBox, Box, Label, FloatSlider, Dropdown

In [2]:
def Tmatrix(τ, μ, format=None):
    tau = τ / μ
    n = len(tau)
    Δτ = numpy.roll(tau, -1) - tau
    Δτm = numpy.roll(Δτ, 1)
    
    A = 2 / (Δτm * (Δτm + Δτ))
    B = 1 + 2 / (Δτ * Δτm)
    C = 2 / (Δτ * (Δτm + Δτ))
    
    # Boundary conditions top
    B[0] = 2 / Δτ[0]**2 + 2 / Δτ[0] + 1
    C[0] = 2 / Δτ[0]**2
    # Boundary conditions bottom
    A[n-1] = 2 / (Δτ[n-2] * (Δτ[n-2] + 2))
    B[n-1] = (2 + 2*Δτ[n-2] + Δτ[n-2]**2) / (Δτ[n-2] * (Δτ[n-2] + 2))
    
    if format == 'banded':  # for use with scipy.linalg.solve_banded
        T = numpy.zeros((3, n))
        T[0, 1:] = -C[:-1]
        T[1] = B
        T[2, :-1] = -A[1:]
        return T
    elif format == 'sparse':  # for use with scipy.sparse.linalg.spsolve
        return diags([-A[1:], B, -C[:-1]], [-1, 0, 1], format='csc')
    else:  # normal matrix
        return diags([-A[1:], B, -C[:-1]], [-1, 0, 1]).toarray()
    
    

def quadrature(k=5):
    """
    Returns the nodes and weights form a Gaussian
    quadrature with k points. Rescaled to an interval
    from [0, 1].
    """
    nodes, weights = roots_legendre(k)
    return nodes/2 + 0.5, weights/2

In [7]:
def scattering_atmos():
    def _compute_radiation(τ, ε, B):
        n = len(τ)
        Λ = numpy.zeros((n, n))
        for mu, w in zip(*quadrature()):
            T = Tmatrix(τ, mu)
            Λ += w * numpy.linalg.inv(T)
        M = numpy.linalg.inv(numpy.eye(n) - (1 - ε) * Λ)
        S = ε * numpy.dot(M, B)
        J = numpy.dot(Λ, S)
        return S, J
    
    eps = 1e-1
    eta = 1e3
    tau = numpy.logspace(-9, 2, 100)
    B = {"Chromosphere": 1 + 0.2 * tau + 18.7 * numpy.exp(-1.44e4 * tau),
         "Photosphere": 1 + 1.5*tau,
         "Constant": numpy.ones_like(tau)}
    B_initial = "Constant"
    
    S, J = _compute_radiation(tau * eta, eps, B[B_initial])
    fig_layout = Layout(align_items='stretch', width='70%')
    fig = plt.figure(title='Coherent scattering in a plane-parallel atmosphere', layout=fig_layout)
    B_plot = plt.plot(tau, B[B_initial], 'b-', scales={'x': LogScale(), 'y': LogScale()})
    S_plot = plt.plot(tau, S, 'r--')
    J_plot = plt.plot(tau, J, 'k:')
    plt.xlabel("τ continuum")
    labels = plt.label(['B', 'S', 'J'], 
                       x=[B_plot.x[0], S_plot.x[0]*2, J_plot.x[0]],
                       y=[B_plot.y[0]* 1.5, S_plot.y[0]*1.5, J_plot.y[0]*1.2],
                       colors=['black'])
    eps_slider = FloatSlider(min=-6, max=0, step=0.02, value=-1, description=r'$\log\varepsilon$')
    eta_slider = FloatSlider(min=0, max=9., step=0.02, value=2, description=r'$\log\eta$')
    B_selector = Dropdown(options=["Chromosphere", "Photosphere", "Constant"],
                          value=B_initial, description='B')
    
    def plot_update(logε=eps, logη=eta, B_shape="Constant"):
        S, J = _compute_radiation(tau * 10**logη, 10**logε, B[B_shape])
        B_plot.y = B[B_shape]
        S_plot.y = S
        J_plot.y = J
        labels.y = [B[B_shape][0]*1.2, S[0]*1.2, J[0]*1.2]

        
    widg = interactive(plot_update, logε=eps_slider, logη=eta_slider, B_shape=B_selector)
    controls = HBox([widg.children[2], widg.children[0], widg.children[1]],
                    layout=Layout(align_items='stretch', width='100%'))
    return Box([controls, fig], layout=Layout(
        display='flex',
        flex_flow='column',
        align_items='stretch',
        width='100%',
    ))
    return VBox([controls, fig], layout=Layout(height='100%'))
    
 

In [8]:
scattering_atmos()

Box(children=(HBox(children=(Dropdown(description='B', index=2, options=('Chromosphere', 'Photosphere', 'Const…